In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/banth-hate-multilabel/banglahatebert-hate-test.csv
/kaggle/input/banth-hate-multilabel/indicbert-hate-test.csv
/kaggle/input/banth-hate-multilabel/banglishbert-hate-test.csv
/kaggle/input/banth-hate-multilabel/charbert-hate-test.csv
/kaggle/input/banth-hate-multilabel/mbert_multilabel_predictions.csv
/kaggle/input/banth-hate-multilabel/mbert-hate-test.csv
/kaggle/input/banth-hate-multilabel/vacbert-hate-test.csv
/kaggle/input/banth-hate-multilabel/tbbert-hate-test.csv
/kaggle/input/banth-hate-multilabel/tbbanglabert-hate-test.csv
/kaggle/input/banth-hate-multilabel/tbmbert-hate-test.csv
/kaggle/input/banth-hate-multilabel/tb-banglishbert-hate-test.csv
/kaggle/input/banth-hate-multilabel/sagorbertbase-hate-test.csv
/kaggle/input/banth-hate-multilabel/cuetbanglabert-hate-test.csv
/kaggle/input/banth-hate-multilabel/indicbert_multilabel_predictions.csv
/kaggle/input/banth-hate-multilabel/muril-hate-test.csv
/kaggle/input/gpt4o-outputs/gpt4o_outputs/multilabel_translation/out

In [2]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, hamming_loss

### binary function

In [3]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

def calculate_binary_metrics(y_true, y_pred):
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    # Calculate metrics with different averaging
    metrics = {
        'F1_Macro': f1_score(y_true, y_pred, average='macro'),
        # 'F1_Micro': f1_score(y_true, y_pred, average='micro'),

        'Accuracy_Macro': accuracy_score(y_true, y_pred),  # For binary, macro and micro are the same
        # 'Accuracy_Micro': accuracy_score(y_true, y_pred),  # Included for consistency
        
        'Precision_Macro': precision_score(y_true, y_pred, average='macro'),
        # 'Precision_Micro': precision_score(y_true, y_pred, average='micro'),
        
        'Recall_Macro': recall_score(y_true, y_pred, average='macro'),
        # 'Recall_Micro': recall_score(y_true, y_pred, average='micro'),
        
        
    }
    
    # Calculate specificity (true negative rate)
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
    metrics['Specificity'] = specificity
    
    # Round all metrics to 4 decimal places
    return {k: round(v, 4) for k, v in metrics.items()}



### multilabel function

In [4]:
def calculate_multilabel_metrics(df, true_columns, pred_columns):
    """
    Calculate multilabel classification metrics for hate speech detection
    
    Parameters:
    df: pandas DataFrame containing the dataset
    true_columns: list of column names for true labels
    pred_columns: list of column names for predicted labels
    
    Returns:
    Dictionary containing all calculated metrics
    """
    # Extract true and predicted labels
    y_true = df[true_columns].values
    y_pred = df[pred_columns].values
    
    # Calculate metrics
    metrics = {
        # 'F1_Micro': f1_score(y_true, y_pred, average='micro'),
        'F1_Macro': f1_score(y_true, y_pred, average='macro'),

        # 'Accuracy_Micro': accuracy_score(y_true.ravel(), y_pred.ravel()),
        'Accuracy_Macro': np.mean([accuracy_score(y_true[:, i], y_pred[:, i]) 
                                  for i in range(y_true.shape[1])]),
        
        # 'Precision_Micro': precision_score(y_true, y_pred, average='micro'),
        'Precision_Macro': precision_score(y_true, y_pred, average='macro'),
        
        # 'Recall_Micro': recall_score(y_true, y_pred, average='micro'),
        'Recall_Macro': recall_score(y_true, y_pred, average='macro'),
        
        'Subset_Accuracy': accuracy_score(y_true, y_pred),
        'Hamming_Loss': hamming_loss(y_true, y_pred)
    }
    
    
    return metrics

## binary

gpt4o_binary_few_shot_cot

In [5]:
df = pd.read_csv("/kaggle/input/gpt4o-outputs/gpt4o_outputs/binary_translation/output/gpt-4o/gpt4o__english_binary_test.csv")
# df
y_true = df['Label']
# # y_pred = df['is_hate'].astype(int)
y_pred = df['is_hate']
calculate_binary_metrics(y_true, y_pred)

{'F1_Macro': 0.7019,
 'Accuracy_Macro': 0.7384,
 'Precision_Macro': 0.6945,
 'Recall_Macro': 0.7216,
 'Specificity': 0.7606}

## Multilabel

In [6]:
true_columns = ['Political', 'Religious', 'Gender', 'Personal Offense', 
                    'Abusive/Violence', 'Origin', 'Body Shaming', 'Misc']
pred_columns = ['political', 'religious', 'gender', 'personal', 
            'abusive_violent', 'origin', 'bodyshaming', 'misc']
pred_columns_sifat = ['Pred_Political', 'Pred_Religious', 'Pred_Gender', 'Pred_Personal Offense', 
                    'Pred_Abusive/Violence', 'Pred_Origin', 'Pred_Body Shaming', 'Pred_Misc']

In [7]:
df = pd.read_csv("/kaggle/input/gpt4o-outputs/gpt4o_outputs/multilabel_translation/output/gpt-4o/gpt4o__bangla_multilabel.csv")

metrics = calculate_multilabel_metrics(df, true_columns, pred_columns)

# Print metrics
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

F1_Macro: 0.3625
Accuracy_Macro: 0.8354
Precision_Macro: 0.3161
Recall_Macro: 0.5332
Subset_Accuracy: 0.2154
Hamming_Loss: 0.1646


In [8]:
df_indic_multi = pd.read_csv("/kaggle/input/banth-hate-multilabel/indicbert_multilabel_predictions.csv")


In [9]:
df_indic_multi.columns

Index(['Text', 'Label', 'Political', 'Religious', 'Gender', 'Personal Offense',
       'Abusive/Violence', 'Origin', 'Body Shaming', 'Misc', 'bangla',
       'english', 'Pred_label', 'Pred_Political', 'Pred_Religious',
       'Pred_Gender', 'Pred_Personal Offense', 'Pred_Abusive/Violence',
       'Pred_Origin', 'Pred_Body Shaming', 'Pred_Misc'],
      dtype='object')

In [10]:
df_indic_multi = df_indic_multi.dropna()
df_indic_com = df_indic_multi[['Pred_label', 'Pred_Political', 'Pred_Religious',
       'Pred_Gender', 'Pred_Personal Offense', 'Pred_Abusive/Violence',
       'Pred_Origin', 'Pred_Body Shaming', 'Pred_Misc']]

In [11]:
df_indic_com.shape

(937, 9)

In [12]:
df_indic_bin = pd.read_csv("/kaggle/input/banth-hate-multilabel/mbert-hate-test.csv")
df_indic_bin = df_indic_bin[df_indic_bin['Pred_label']==1]
df_indic_bin

,Text,Label,Political,Religious,Gender,Personal Offense,Abusive/Violence,Origin,Body Shaming,Misc,bangla,english,Pred_label
3,Myb eta India cal parar meye,1,0,0,0,1,0,1,0,0,মেইবি এটা ইন্ডিয়া চাল পাড়ার মেয়ে,This is India rice girl.,1
9,Chup kor. Shongshod vobone student der lash pa...,0,0,0,0,0,0,0,0,0,চুপ কর।শংশদ ভয়েন স্টুডেন্ট দের চাবুক পাওয়া গ...,Shut up. The flogging of the Voyan students i...,1
15,Tur ma deshe asar sate sate ato gula kuner bic...,1,0,0,0,1,0,0,0,0,তুর মা দেশে আসার সাথে সাথে এত গুলা কানের মাশুল...,Your mother's arrival will cost you so many e...,1
26,Otey juta peta kortey kortey hospital thekey b...,1,0,0,0,0,1,0,0,0,ওইটায় জুতা পেটা করতে করতে হাসপাতাল থেকে বের ক...,Get her out of the hospital with shoes and li...,1
35,Etai hosse bokacoda magir digital desh.....,1,0,0,0,0,1,0,0,0,এটাই হচ্ছে বোকাচোদা ম্যাজিক ডিজিটাল দেশ.....,This is the stupid magic digital country...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3718,policer mare sody,1,0,0,0,1,0,0,0,0,পুলিশ মারে সোডি,Police Mare Sody,1
3719,Karap meye ekta,1,0,0,0,1,0,0,0,0,করপ মেয়ে একটা,She's a woman.,1
3720,"Sala osir kolija ki duita, take gonodulai deya...",1,0,0,0,0,1,0,0,0,"শালা ওসির কুলিজা কি দুইটা, তাকে গনদুলাই দেয়া...","What the hell is OC's Kulija, let's get him a...",1
3727,This is Gopali.......🍌🍌🍌🍌,0,0,0,0,0,0,0,0,0,দিস ইজ গোপালি.....🍌🍌🍌🍌,This is Gopali...🍌🍌🍌🍌,1
